![1](images/FEMTO_FOREST_400_1.png)
    
![2](images/FEMTO_FOREST_400_2.png)

![3](images/FEMTO_FOREST_400_3.png)
    
<!-- ![4](images/FEMTO_FOREST_400_4.png) -->

In [1]:
import json
import pennylane as qml
import pennylane.numpy as np

# Write any helper functions you need
def U_matrix(a):
    return np.array([[a, np.sqrt(1-(a**2))],[np.sqrt(1-(a**2)), -a]])

def coeffs(y1, y2):
    x = np.linspace(0,1,5)
    fit1 = np.polyfit(x,y1,3)
    fit2 = np.polyfit(x,y2,3)
    alpha= fit1[0]+fit2[0]*1j
    beta = fit1[2]+fit2[2]*1j
    return alpha, beta
def coefficients(angles):
    
    """ This function returns the coefficients associated with the polynomial generated by 
    the QSP routine as a function of the phase angles.
    
    Args:
        - angles (np.array(float)): Array of real numbers containing the four phase angles,
        in reverse order of application.
    
    Returns:       
        - (np.array(complex)): A numpy array containing the coefficients of the polynomial
        generated by QSP, where the first element is the coefficient of the cubic term and
        the second element is for the linear term.
    """
    

    # Put your code here
    def circuit(a):
        qml.RZ(-2*angles[3], 0)
        qml.QubitUnitary(U_matrix(a), 0)
        qml.RZ(-2*angles[2], 0)
        qml.QubitUnitary(U_matrix(a), 0)
        qml.RZ(-2*angles[1], 0)
        qml.QubitUnitary(U_matrix(a), 0)
        qml.RZ(-2*angles[0], 0)
            
    matrix_fn = qml.matrix(circuit)
    y1 = []
    y2 = []
    for i in np.linspace(0,1,5):
        y1.append(matrix_fn(i)[0][0].real)
        y2.append(matrix_fn(i)[0][0].imag)
    
    return coeffs(y1, y2)
# These functions are responsible for testing the solution.


def run(test_case_input: str) -> str:
    ins = np.array(json.loads(test_case_input))
    coeffs = coefficients(ins)
    outs = [[np.real(elem).numpy(), np.imag(elem).numpy()] for elem in coeffs]
    
    return str(outs)


def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)

    assert np.allclose(solution_output, expected_output, rtol = 1e-4)
